In [ ]:
import matplotlib.pyplot as plt

import numpy as np #importa a biblioteca usada para trabalhar com vetores e matrizes
import pandas as pd #importa a biblioteca usada para trabalhar com dataframes
import util
from random import randrange
import math
#importa o arquivo e extrai as features
Xfeatures, Y = util.extract_features('datasets/ARCHEAGE.csv')

In [ ]:
def get_split(X, Y, n_features):
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    atr = []
    while len(atr) < n_features:
        idx = randrange(n_features)
        if idx not in atr:
            atr.append(idx)
        for idx in atr:
            for row in X:
                groups = test_split(idx, row[idx], X, Y)
                gini = gini_index(groups, Y)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = idx, row[idx], gini, groups
            return {'index':b_index, 'value':b_value, 'groups':b_groups}

In [ ]:
def test_split(idx, value, X, Y):
    left, right = [], []
    i = 0
    for row in X:
        if row[idx] < value:
            np.append(row, Y[i])
            left.append(row)
            i += 1
        else:
            np.append(row, Y[i])
            right.append(row)
            i += 1
    return left, right

In [ ]:
def gini_index(groups, classes):
    n = sum([len(group) for group in groups])
    gini = 0.0
    
    for group in groups:
        size = len(group)
        
        if size == 0:
            continue
        score = 0.0
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        gini += (1.0 - score) * (size / float(n))
        return gini

In [ ]:
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

In [ ]:
def split(node, max_depth, min_size, n_features, depth):
    left, right = node['groups']
    del(node['groups'])
    
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, n_features)
        split(node['left'], max_depth, min_size, n_features, depth+1)
        
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth+1)

In [ ]:
def build_tree(X_train, max_depth, min_size, n_features, Y_train):
    root = get_split(X_train, Y_train, n_features)
    split(root, max_depth, min_size, n_features, 1)
    return root

In [ ]:
def subsample(X, ratio):
    sample = list()
    n_sample = round(len(X) * ratio)
    while len(sample) < n_sample:
        idx = randrange(len(X))
        sample.append(X[idx])
    return sample

In [ ]:
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [ ]:
def bagging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    return max(set(predictions), key=predictions.count)

In [ ]:
def random_forest(X_train, X_test, max_depth, min_size, sample_size, n_trees, n_features, Y_train):
    trees = []
    for i in range(n_trees):
        sample = subsample(X_train, sample_size)
        tree = build_tree(sample, max_depth, min_size, n_features, Y_train)
        trees.append(tree)
    predictions = [bagging_predict(trees, row) for row in X_test]
    return(predictions)

In [ ]:
# semente usada na randomizacao dos dados.
randomSeed = 10 

# gera os indices aleatorios que irao definir a ordem dos dados
idx_perm = np.random.RandomState(randomSeed).permutation(range(len(Y)))

# ordena os dados de acordo com os indices gerados aleatoriamente
Xk, Yk = Xfeatures[idx_perm, :], Y[idx_perm]

nFolds = 10
classes = [0,1]
folds = util.stratified_kfolds(Yk, nFolds, classes)

k=1
resultados=[]

for train_index, test_index in folds:
    print('\n-----------\n%d-fold: \n-----------\n' % (k) )
    
    # se train_index ou test_index forem vazios, interrompe o laco de repeticao
    if len(train_index)==0 or len(test_index)==0: 
        print('\tErro: o vetor com os indices de treinamento ou o vetor com os indices de teste esta vazio')      
        break
        
    totalFold = len(train_index)+len(test_index)

    X_train, X_test = Xk[train_index, :], Xk[test_index, :];
    Y_train, Y_test = Yk[train_index], Yk[test_index];
    
    # separa os dados de treinamento em treinamento e validacao
    pTrain = 0.8
    train_index_v, val_index = util.stratified_holdOut(Y_train, pTrain)
    
    max_depth = 10
    min_size = 1
    sample_size = 1.0
    n_features = int(math.sqrt(len(Xfeatures[0])))
    for n_trees in [1, 5, 10]:
        Y_pred = random_forest(X_train, X_test, max_depth, min_size, sample_size, n_trees, len(Xfeatures[0]), Y_train)
        print('Trees: %d' % n_trees)
        cm = util.get_confusionMatrix(Y_test, Y_pred, classes)
        print(cm)
        print('\n\tResultado no fold atual usando o melhor parametro encontrado:')
        auxResults = util.relatorioDesempenho(cm, classes, imprimeRelatorio=True)
        resultados.append( auxResults )
        
    k+=1